In [195]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
# ignore warnings
import warnings
import math

warnings.filterwarnings('ignore')

In [196]:
column_names = [
"address",
"price",
"gross_tax",
"strata_maintenance_fees",
"bedrooms",
"bathrooms",
"property_type",
"property_age",
"title",
"style",
"heating_type",
"feature",
"amenities",
"appliances",
"community",
"days_on_rew",
"property_views",
"mls®_number",
"source",
'frontage', 
'lot_size', 
'year_built', 
'depth',
'half_bathrooms'
]



In [197]:

def get_df(file_path, apply_columns=True):

    sys.path.append(file_path)

    data = pd.read_csv(file_path, on_bad_lines='skip')

    # data.dropna(inplace=True)
    if apply_columns:
        data.columns = column_names
    return data



folder = "../data/raw_2023_04_19/"

filenames = {
    "burnaby": "burnaby_real_estate_data.csv",
    "langley":"langley_real_estate_data.csv",
    "richmond":"richmond_real_estate_data.csv",
    "maple_ridge":"maple-ridge_real_estate_data.csv",
    "surrey":"surrey_real_estate_data.csv",
    "new_west": "new-westminster_real_estate_data.csv",
    "vancouver":"vancouver_real_estate_data.csv",
    "coquitlam":"coquitlam_real_estate_data.csv",
    "north_vancouver":"north-vancouver_real_estate_data.csv",
    "west_vancouver":"west-vancouver_real_estate_data.csv",
    "delta":"delta_real_estate_data.csv",
    "pitt_meadows":"pitt-meadows_real_estate_data.csv",
    "kelowna":"kelowna_real_estate_data.csv",
    "port_coquitlam":"port-coquitlam_real_estate_data.csv"
}

dataframes = {}

for k,v in filenames.items():
    dataframes[k] = get_df(folder + v)

dataframes.keys()



dict_keys(['burnaby', 'langley', 'richmond', 'maple_ridge', 'surrey', 'new_west', 'vancouver', 'coquitlam', 'north_vancouver', 'west_vancouver', 'delta', 'pitt_meadows', 'kelowna', 'port_coquitlam'])

In [198]:
# get neighbourhoods per city

neighbourhoods_city = {}

for city, df in dataframes.items():
    
    neighbourhoods = df['community'].unique()
    for neighbourhood in neighbourhoods:

        neighbourhoods_city[neighbourhood] = city


In [199]:
# find out which city the row is in
clean_combined_data = get_df(file_path=folder + "clean_combined_data.csv", apply_columns=False)

clean_combined_data['city'] = clean_combined_data['community'].apply(lambda x: neighbourhoods_city.get(x, 'unknown'))

# Select Features

In [212]:
# Select the specified columns
# selected_columns = ['address', 'price', 'gross_tax', 'bedrooms',
#                     'bathrooms', 'property_type', 
#                     'community', 'lot_size', 'half_bathrooms', 'lot_width', 'lot_length',
#                     'age', 'bungalow', 'storey', 'basement', 'laneway_house', 'garage',
#                     'split_entry', 'city']

features = [
    'price', 
    'city',
    'gross_tax',
    'community', 
    'bedrooms',
    'bathrooms', 
    'lot_width',
    'lot_length',
    'age',
    'bungalow',
    'storey', 
    'basement', 
    'laneway_house',
    'garage',
    'split_entry',
]

df = clean_combined_data[features]
df.columns



Index(['price', 'city', 'gross_tax', 'community', 'bedrooms', 'bathrooms',
       'lot_width', 'lot_length', 'age', 'bungalow', 'storey', 'basement',
       'laneway_house', 'garage', 'split_entry'],
      dtype='object')

# Code Cleanup

In [201]:
# Drop 0 Rows
# df = df[df['gross_tax'] != 0]
# df = df[df['lot_size'] != 0]
df = df[df['lot_width'] != 0]
df = df[df['lot_length'] != 0]
# before (2104, 294)
df = df.dropna()
# after (1379, 294)

# round price for better prediction
# df['price'] = df['price'].apply(lambda x: math.ceil(x / 500000) * 500000)

df.shape


(1382, 15)

In [219]:
pd.set_option('max_colwidth', 1000)

In [221]:
# Identify the categorical columns
categorical_columns = df.select_dtypes(include=['object']).columns

# Apply one-hot encoding to the categorical columns
df = df.drop_duplicates()

df = pd.get_dummies(df, columns=categorical_columns)
# set max number of columns and rows to display


pd.set_option('max_colwidth', 1000)
df.style.set_properties(subset=None, **{'width': '200px'})

print(df.columns)
arr = []
for feature in df.columns:
    arr.append(feature)

print(arr)

Index(['price', 'gross_tax', 'bedrooms', 'bathrooms', 'lot_width',
       'lot_length', 'age', 'bungalow', 'storey', 'basement',
       ...
       'community_whitby estates', 'community_white rock',
       'community_whonnock', 'community_whytecliff',
       'community_willingdon heights', 'community_willoughby heights',
       'community_woodland acres', 'community_woodlands-sunshine-cascade',
       'community_woodwards', 'community_yaletown'],
      dtype='object', length=299)
['price', 'gross_tax', 'bedrooms', 'bathrooms', 'lot_width', 'lot_length', 'age', 'bungalow', 'storey', 'basement', 'laneway_house', 'garage', 'split_entry', 'city_burnaby', 'city_coquitlam', 'city_delta', 'city_kelowna', 'city_langley', 'city_maple_ridge', 'city_new_west', 'city_north_vancouver', 'city_pitt_meadows', 'city_port_coquitlam', 'city_richmond', 'city_surrey', 'city_vancouver', 'city_west_vancouver', 'community_albion', 'community_aldergrove', 'community_altamont', 'community_ambleside', 'community

In [203]:
# Export training data
df.to_csv('../data/training_data_april.csv', index=False)


# Neural Network

## Training

In [204]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import math
import tensorflow as tf
import keras


data = df

# Define the target variable (price) and feature columns
target = 'price'
features = data.drop(columns=[target]).columns

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2, random_state=42)

# Normalize the data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# lstm
# Define the neural network model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=500, batch_size=32, verbose=1, callbacks=[keras.callbacks.EarlyStopping(patience=3)])


Epoch 1/500
35/35 [==============================] - 0s 1ms/step - loss: 15685578129408.0000 - accuracy: 0.0000e+00
Epoch 2/500
35/35 [==============================] - 0s 894us/step - loss: 15684475027456.0000 - accuracy: 0.0000e+00
Epoch 3/500
35/35 [==============================] - 0s 1ms/step - loss: 15657718513664.0000 - accuracy: 0.0000e+00
Epoch 4/500
35/35 [==============================] - 0s 1ms/step - loss: 15369605480448.0000 - accuracy: 0.0000e+00
Epoch 5/500
35/35 [==============================] - 0s 1ms/step - loss: 13809198563328.0000 - accuracy: 0.0000e+00
Epoch 6/500
35/35 [==============================] - 0s 952us/step - loss: 9510347341824.0000 - accuracy: 0.0000e+00
Epoch 7/500
35/35 [==============================] - 0s 972us/step - loss: 5859180544000.0000 - accuracy: 0.0000e+00
Epoch 8/500
35/35 [==============================] - 0s 980us/step - loss: 5191433715712.0000 - accuracy: 0.0000e+00
Epoch 9/500
35/35 [==============================] - 0s 974us/step 

In [205]:
print(data.shape)
print(X_train.shape)
print(X_train_scaled.shape)
X_train_scaled[0].shape

(1382, 268)
(1105, 267)
(1105, 267)


(267,)

## Evaluation

In [206]:

# Evaluate the model
loss = model.evaluate(X_test_scaled, y_test)
mse = loss[0]
formatted_mse = "{:,.2f}".format(mse)
print(f'Mean Squared Error: {formatted_mse}')

rmse = math.sqrt(mse)
formatted_mse = "{:,.2f}".format(rmse)
print(f'Root Mean Squared Error: {formatted_mse}')
# Root Mean Squared Error: 1,226,026.20

9/9 [==============================] - 0s 698us/step - loss: 1524868579328.0000 - accuracy: 0.0000e+00
Mean Squared Error: 1,524,868,579,328.00
Root Mean Squared Error: 1,234,855.69


## Predict

In [207]:
X_test_scaled.shape
# prediction = model.predict(X_test_scaled[0:])
# prediction

(277, 267)

In [208]:

# Make predictions
predictions = model.predict(X_test_scaled)

9/9 [==============================] - 0s 605us/step


In [209]:
# Concatenate predicted and actual values
results = np.concatenate((predictions, y_test.values.reshape(-1, 1)), axis=1)

# Create DataFrame to show it side by side with predicted, actual and their differences
results_df = pd.DataFrame(results, columns=['Predicted', 'Actual'])
results_df['Difference'] = abs(results_df['Actual'] - results_df['Predicted'])

formated_df = results_df.copy()
formated_df['Predicted'] = results_df['Predicted'].apply(lambda x: '{:,.2f}'.format(x))
formated_df['Actual'] = results_df['Actual'].apply(lambda x: '{:,.2f}'.format(x))
formated_df['Difference'] = results_df['Difference'].apply(lambda x: '{:,.2f}'.format(x))

mean = results_df['Difference'].mean()
formatted_mean = "{:,.2f}".format(mean)
print("average difference",  formatted_mean)
# Print results
formated_df.head()


average difference 808,323.13


,Predicted,Actual,Difference
0,"2,659,563.50","2,499,999.00","159,564.50"
1,"1,843,018.88","2,379,312.00","536,293.12"
2,"2,287,282.50","3,300,000.00","1,012,717.50"
3,"1,393,322.38","1,798,800.00","405,477.62"
4,"1,798,154.38","1,899,000.00","100,845.62"


In [210]:
# Concatenate predicted and actual values
results = np.concatenate((predictions, y_test.values.reshape(-1, 1)), axis=1)

# Create DataFrame
results_df = pd.DataFrame(results, columns=['Predicted', 'Actual'])

# Print results
results_df.head()

,Predicted,Actual
0,2659563.500,2499999.0
1,1843018.875,2379312.0
2,2287282.500,3300000.0
3,1393322.375,1798800.0
4,1798154.375,1899000.0


In [211]:
model.save('housingNNmodel')

INFO:tensorflow:Assets written to: housingNNmodel/assets
